In [72]:
"""
Movie recommendation system based on genres.
"""

'\nMovie recommendation system based on genres.\n'

In [73]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [74]:
# Load movie dataset (Nomainīts, jo dati autoram glabājās lokāli)
movies = pd.read_csv("C:\Bakalaurs_praktiskais\Bakalaura-darbs\movies.csv")
ratings = pd.read_csv(r"C:\Bakalaurs_praktiskais\Bakalaura-darbs\ratings.csv")

In [75]:
# Data preprocessing
movies['genres'] = movies['genres'].apply(lambda x: x.lower().replace('|', ' '))

In [76]:
# Create TF-IDF vectorizer and fit the movie dataset
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(movies['genres'])

In [77]:
#Calculate similarity matrix
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [78]:
def find_movie_index(movie_title, year):
    if year != None:
        movie_title = f"{movie_title} ({year})"
        if movie_title not in movies['title'].values:
            return None
        return movies[movies['title'] == movie_title].index[0]
    else:
        if not any(movies['title'].str.contains(movie_title)):
            return None
        return movies[movies['title'].str.contains(movie_title)].index[0]

In [79]:
def recommend_movies(movie_title, year=None, n_recommendations=5):
    movie_index = find_movie_index(movie_title, year)
    if movie_index is None:
        movie_not_found_message = f"Movie  '{movie_title}' not found in dataset."
        print(movie_not_found_message)
        return None

    sim_scores = list(enumerate(cosine_sim_matrix[movie_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n_recommendations + 1]

    recommended_movie_indices = [i[0] for i in sim_scores]
    return movies.iloc[recommended_movie_indices]

In [80]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

def evaluate(movies, ratings, recommend_function, n_recommendations=5, top_n_users=20):
    # Apvieno filmu vērtējumus ar filmām
    data = pd.merge(ratings, movies, on='movieId')
    
    # Izvēlās tikai aktīvākos lietotājus
    top_users = data['userId'].value_counts().head(top_n_users).index.tolist()
      # Izveido tukšus sarakstus priekš metrikām
    all_precisions, all_recalls, all_f1s = [], [], []
    user_maes, user_rmses = [], []
    all_actual_ratings = []
    all_predicted_ratings = []
    user_metrics = []
    
    for user_id in top_users:
        user_ratings = data[data['userId'] == user_id]
         # Norāda vērtējumu robežu, kuru uzskatīt par pozitīvi novērtētu filmu
        liked_movies = user_ratings[user_ratings['rating'] >= 3.5]
        
        # Izlaiž lietotājus, kuri ir vērtējuši mazāk par 5 filmām
        if len(liked_movies) < 5:
            continue
        
           # Uz nejaušību izvēlās vienu filmu, kura ir vērtēta pozitīvi no ietiekumiem
        seed_row = liked_movies.sample(1).iloc[0]
        seed_title = seed_row['title']
        
        # Apstrādā datus tā, lai nosaukumā netiek iekļauts gads "Title (Year)"
        try:
            if seed_title.strip()[-1] == ')':
                year = int(seed_title.strip()[-5:-1])
                title = seed_title[:-7]  # Noņem gadu no nosaukuma " (YYYY)"
            else:
                year = None
                title = seed_title
        except:
            year = None
            title = seed_title
        
        # Ģenerē ieteikumus, balstoties uz "seed" filmu
        recs = recommend_function(title, year, n_recommendations=n_recommendations)
        if recs is None:
            continue
         # Ieteikumu saraksts, ko atgriež ieteikumu sistēma
        recommended_titles = set(recs['title'].tolist())
        actual_liked_titles = set(liked_movies['title'].tolist()) - {seed_title}
        
        # Aprēķina atbilstošas filmas, kuras ir ieteiktas
        hits = len(recommended_titles & actual_liked_titles)
        
        # Aprēķina precizitāti, atsaikumu un f1 mērījumu
        precision = hits / len(recommended_titles) if recommended_titles else 0
        recall = hits / len(actual_liked_titles) if actual_liked_titles else 0
        if precision + recall > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
        else:
            f1 = 0
        
        all_precisions.append(precision)
        all_recalls.append(recall)
        all_f1s.append(f1)
        
         # MAE & RMSE aprēķins (salīdzina tikai filmas, kurām ir patiesie vērtējumi)
        intersect_titles = recommended_titles & set(user_ratings['title'].tolist())
        if intersect_titles:
            actual_ratings = user_ratings[user_ratings['title'].isin(intersect_titles)]['rating'].values
            predicted_ratings = np.array([4.0] * len(actual_ratings)) # Norāda vērtējumu robežu, kuru uzskatīt par positīvi novērtētu
            
            
            all_actual_ratings.extend(actual_ratings)
            all_predicted_ratings.extend(predicted_ratings)
            
            mae = mean_absolute_error(actual_ratings, predicted_ratings)
            rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))
            
            user_maes.append(mae)
            user_rmses.append(rmse)
            
            user_metrics.append({
                'userId': user_id,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'mae': mae,
                'rmse': rmse,
                'rated_count': len(actual_ratings)
            })
    
    overall_mae = None
    overall_rmse = None
    if all_actual_ratings:
        overall_mae = mean_absolute_error(all_actual_ratings, all_predicted_ratings)
        overall_rmse = np.sqrt(mean_squared_error(all_actual_ratings, all_predicted_ratings))
    
    print("\n--- Novērtējums ---")
    print(f"Novērtēto lietotāju skaits: {len(all_precisions)}")
    print(f"Vidējā Precizitāte@{n_recommendations}: {np.mean(all_precisions):.4f}")
    print(f"Videājais Atsaukums@{n_recommendations}: {np.mean(all_recalls):.4f}")
    print(f"Vidējais F1@{n_recommendations}: {np.mean(all_f1s):.4f}")
    
    # Izprintē vidējo lietotāju MAE un RMSE (pa lietotājiem)
    if user_maes and user_rmses:
        print(f"Vidējais lietotāju MAE: {np.mean(user_maes):.4f}")
        print(f"Vidējais lietotāju RMSE: {np.mean(user_rmses):.4f}")
    else:
        print("Lietotāju MAE/RMSE nevar aprēķināt (nav pietiekamu datu).")
    
    # Izprintē kopējo MAE un RMSE (visiem vērtējumiem kopā)
    if overall_mae is not None and overall_rmse is not None:
        print(f"Kopējais MAE: {overall_mae:.4f}")
        print(f"Kopējais RMSE: {overall_rmse:.4f}")
    else:
        print("Kopējo MAE/RMSE nevar aprēķināt (nav pietiekamu datu).")
    
    # Izprintē tabulu ar lietotāju metrikām
    print("\n--- Detalizēta lietotāju statistika ---")
    print(f"{'LietotājaId':<10} {'Precizitāte':>10} {'Atsaukums':>10} {'F1':>10} {'MAE':>10} {'RMSE':>10} {'Atbilstošo filmu skaits ieteikumu sarakstā':>10}")
    print(f"{'-'*70}")
    
    # Sakārto pēc lietotāja ID
    user_metrics.sort(key=lambda x: x['userId'])
    for metrics in user_metrics:
        print(f"{metrics['userId']:<10} {metrics['precision']:>10.4f} {metrics['recall']:>10.4f} {metrics['f1']:>10.4f} {metrics['mae']:>10.4f} {metrics['rmse']:>10.4f} {metrics['rated_count']:>10}")
    
    # Atgriež visas metrikas kā vārdnīcu
    return {
        'precision': np.mean(all_precisions),
        'recall': np.mean(all_recalls),
        'f1': np.mean(all_f1s),
        'lietotāja_mae': np.mean(user_maes) if user_maes else None,
        'lietotāja_rmse': np.mean(user_rmses) if user_rmses else None,
        'kopējais_mae': overall_mae,
        'kopējais_rmse': overall_rmse,
        'lietotāja_metrikas': user_metrics
    }

In [108]:
if __name__ == "__main__":
    year = 1995
    movie_title = "Toy Story"

    user_counts = [10, 20, 50]
    recommendation_counts = [5, 20, 50]

    for top_n_users in user_counts:
        for n_recommendations in recommendation_counts:
            print(f"\nEvaluating with top_n_users={top_n_users}, n_recommendations={n_recommendations}")
            results = evaluate(movies, ratings, recommend_movies,
                               n_recommendations=n_recommendations,
                               top_n_users=top_n_users)
            print(f"Results: Precision={results['precision']:.4f}, Recall={results['recall']:.4f}, F1={results['f1']:.4f}")


Evaluating with top_n_users=10, n_recommendations=5

--- Novērtējums ---
Novērtēto lietotāju skaits: 10
Vidējā Precizitāte@5: 0.0800
Videājais Atsaukums@5: 0.0006
Vidējais F1@5: 0.0011
Vidējais lietotāju MAE: 0.8667
Vidējais lietotāju RMSE: 0.8989
Kopējais MAE: 0.9500
Kopējais RMSE: 1.0607

--- Detalizēta lietotāju statistika ---
LietotājaId Precizitāte  Atsaukums         F1        MAE       RMSE Atbilstošo filmu skaits ieteikumu sarakstā
----------------------------------------------------------------------
68             0.0000     0.0000     0.0000     0.5000     0.5000          1
380            0.0000     0.0000     0.0000     1.0000     1.0000          1
414            0.2000     0.0007     0.0014     1.3333     1.4142          3
474            0.2000     0.0007     0.0015     0.5000     0.5000          2
599            0.4000     0.0042     0.0083     1.0000     1.0801          3
Results: Precision=0.0800, Recall=0.0006, F1=0.0011

Evaluating with top_n_users=10, n_recommendatio